In [7]:
# Librerias
from pathlib import Path
import pandas as pd
import numpy as np
import bcchapi

# ✅ Mostrar todas las columnas al imprimir DataFrames
#pd.set_option('display.max_columns', None)

# Mostrar todas las filas sin truncar
#pd.set_option('display.max_rows', None)


# === FECHA DE CONSULTA ===
fecha_reporte = pd.to_datetime("2025-11-14")

### Contruccion del reporte RDC01 para el patrimonio --->INCOFIN-ACFIN

In [8]:
# 1️ Definir la ruta base hasta la carpeta de trabajo
ruta_cartera = Path.home() / 'EF Securitizadora' / 'Operaciones - Documentos' / 'Documentos' / 'Operaciones' / 'DATOS_PYTHON_BI_SQL' / 'REPORTES_INCOFIN' / '04_saldo_cartera'

# 2️ Ruta completa del archivo
archivo_cartera = ruta_cartera / 'saldo_cartera_20251110.xlsx'

# 3️ Leer el CSV
df_cartera = pd.read_excel(archivo_cartera) 

# 4 Eliminar guion del RUT
df_cartera['rut_id'] = df_cartera['rut_id'].astype(str).str.replace('-', '', regex=False)

# 5 Crear columna TIPO_PERSONA sin dígito verificador
df_cartera['TIPO_PERSONA'] = df_cartera['rut_id'].str[:-1]

# 6 Convertir a número (ignorando errores por seguridad)
df_cartera['TIPO_PERSONA'] = pd.to_numeric(df_cartera['TIPO_PERSONA'], errors='coerce')

# 7 Clasificar: 1 = Persona Natural / 2 = Persona Jurídica
df_cartera['TIPO_PERSONA'] = df_cartera['TIPO_PERSONA'].apply(
    lambda x: 1 if x < 50000000 else 2
)



# ✅ Resultado final
#df_cartera['origen'].value_counts()

#coneccion con api del banco para obetener la informacion al dia del reporte

In [9]:
import sys

ruta_BC = (
    Path.home()
    / "EF Securitizadora"
    / "Operaciones - Documentos"
    / "Documentos"
    / "Operaciones"
    / "REDEC"
)

if str(ruta_BC) not in sys.path:
    sys.path.append(str(ruta_BC))

from BANCO_CENTRAL.banco_central import obtener_valores_bc

print("Módulo importado OK.")

valor_usd, valor_uf, df_bc = obtener_valores_bc(fecha_reporte)

print("USD:", valor_usd)
print("UF:", valor_uf)

Módulo importado OK.
USD: 925.63
UF: 39643.59


# ruta_cuadratura

In [10]:
# 1️⃣ Ruta base y nombre del archivo
ruta_cuadratura = (
    Path.home()
    / 'EF Securitizadora'
    / 'Operaciones - Documentos'
    / 'Documentos'
    / 'Operaciones'
    / 'DATOS_PYTHON_BI_SQL'
    / 'REPORTES_INCOFIN'
    / '00_cuadratura_detalle'
)
archivo_cuadratura = ruta_cuadratura / 'cuadratura_detalle_20251110.xlsx'

# 2️⃣ Leer el archivo Excel
df_cuadratura = pd.read_excel(archivo_cuadratura)

# 3️⃣ Filtrar las filas donde capital e interes son ambos cero
df_cuotones_eliminados = df_cuadratura[
    (df_cuadratura['capital'] == 0) & (df_cuadratura['interes'] == 0)
].copy()

# 4️⃣ Eliminar esas filas del DataFrame original
df_cuadratura = df_cuadratura[
    ~((df_cuadratura['capital'] == 0) & (df_cuadratura['interes'] == 0))
].copy()

# ✅ Resultado: df_cuadratura limpio y df_cuotones_eliminados con los registros descartados
print(f"Filas eliminadas: {len(df_cuotones_eliminados)}")
print(f"Filas restantes: {len(df_cuadratura)}")

# Filtrar df_cuadratura según las operaciones presentes en df_cartera
operaciones_validas = df_cartera["operacion"].unique()
df_filtrado = df_cuadratura[df_cuadratura["operacion"].isin(operaciones_validas)]

# Mostrar resultado / archivo que se encuentra informacion seleccionada a nuestras operaciones
df_filtrado.head()

Filas eliminadas: 120
Filas restantes: 33993


,operacion,cuota,fecha_venc,capital,interes,valor_cuota,fecha_cesion,origen,saldo_insoluto,moneda,...,interes_pagado,cuota_pagada,cesion_informada,dif_capital,saldo_ef,estado_pago,estado_incofin,cuadratura,estado_mora,cap_val
0,10012,1,2021-06-15,5693.71,3993.35,9687.06,2025-11-10,operativo,450688.74,USD,...,0.0,0.0,NaT,0.0,74531.96,CANCELADA,CANCELADA,CUADRADO,AL DÍA,0.0
1,10012,2,2021-07-15,5743.53,3943.53,9687.06,2025-11-10,operativo,444945.21,USD,...,0.0,0.0,NaT,0.0,74531.96,CANCELADA,CANCELADA,CUADRADO,AL DÍA,0.0
2,10012,3,2021-08-15,5793.79,3893.27,9687.06,2025-11-10,operativo,439151.42,USD,...,0.0,0.0,NaT,0.0,74531.96,CANCELADA,CANCELADA,CUADRADO,AL DÍA,0.0
3,10012,4,2021-09-15,5844.48,3842.58,9687.06,2025-11-10,operativo,433306.94,USD,...,0.0,0.0,NaT,0.0,74531.96,CANCELADA,CANCELADA,CUADRADO,AL DÍA,0.0
4,10012,5,2021-10-15,5895.62,3791.44,9687.06,2025-11-10,operativo,427411.32,USD,...,0.0,0.0,NaT,0.0,74531.96,CANCELADA,CANCELADA,CUADRADO,AL DÍA,0.0


In [11]:
# 1️⃣ Asegurar fechas en formato datetime
df_filtrado["fecha_venc"] = pd.to_datetime(df_filtrado["fecha_venc"])
#fecha_reporte = pd.to_datetime("2025-11-14")

# 2️⃣ Calcular interes_diario y dias_devengo
df_filtrado["interes_diario"] = df_filtrado["interes"] / 30
df_filtrado["dias_devengo"] = (df_filtrado["fecha_venc"] - fecha_reporte).dt.days.clip(lower=0)

# 3️⃣ Calcular saldo_insoluto_actual
df_filtrado["saldo_insoluto_actual"] = 0
for op, grupo in df_filtrado.groupby("operacion"):
    filtro = grupo[(grupo["estado_mora"] == "AL DÍA") & (grupo["fecha_venc"] <= fecha_reporte)]
    if not filtro.empty:
        fila_objetivo = filtro.sort_values("fecha_venc").iloc[-1]
        idx = fila_objetivo.name
        df_filtrado.at[idx, "saldo_insoluto_actual"] = fila_objetivo["saldo_insoluto"]

# 4️⃣ Crear df_monto_actual desde operaciones con saldo_insoluto_actual > 0
df_monto_actual = (
    df_filtrado[df_filtrado["saldo_insoluto_actual"] > 0]
    .sort_values("fecha_venc")
    .groupby("operacion", as_index=False)
    .last()[["operacion", "saldo_insoluto_actual"]]
)

# 5️⃣ Agregar dias_devengo e interes_diario desde la siguiente cuota por vencer
siguientes_cuotas = (
    df_filtrado[df_filtrado["fecha_venc"] > fecha_reporte]
    .sort_values(["operacion", "fecha_venc"])
    .groupby("operacion", as_index=False)
    .first()[["operacion", "dias_devengo", "interes_diario"]]
)

df_monto_actual = df_monto_actual.merge(siguientes_cuotas, on="operacion", how="left")

# 6️⃣ Agregar suma de cuotas vencidas por operación (estado_mora == 'MOROSA')
suma_morosidad = (
    df_filtrado[df_filtrado["estado_mora"].str.upper().str.strip() == "MOROSA"]
    .groupby("operacion", as_index=False)["valor_cuota"]
    .sum()
    .rename(columns={"valor_cuota": "suma_cuotas_vencida"})
)

df_monto_actual = df_monto_actual.merge(suma_morosidad, on="operacion", how="left")
df_monto_actual["suma_cuotas_vencida"] = df_monto_actual["suma_cuotas_vencida"].fillna(0)

# Reemplazar NaN en otras columnas por 0
df_monto_actual["dias_devengo"] = df_monto_actual["dias_devengo"].fillna(0)
df_monto_actual["interes_diario"] = df_monto_actual["interes_diario"].fillna(0)

# Calcular monto_actual_con_devengo
df_monto_actual["monto_actual_con_devengo"] = (
    df_monto_actual["saldo_insoluto_actual"] +
    (df_monto_actual["dias_devengo"] * df_monto_actual["interes_diario"]) +
    df_monto_actual["suma_cuotas_vencida"]
)

C:\Users\MarceloVidal\AppData\Local\Temp\ipykernel_12880\3014099305.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["fecha_venc"] = pd.to_datetime(df_filtrado["fecha_venc"])
C:\Users\MarceloVidal\AppData\Local\Temp\ipykernel_12880\3014099305.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["interes_diario"] = df_filtrado["interes"] / 30
C:\Users\MarceloVidal\AppData\Local\Temp\ipykernel_12880\3014099305.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [ ]:
df_filtrado[df_filtrado['operacion'] == 9988 ]

,operacion,cuota,fecha_venc,capital,interes,valor_cuota,fecha_cesion,origen,saldo_insoluto,moneda,...,dif_capital,saldo_ef,estado_pago,estado_incofin,cuadratura,estado_mora,cap_val,interes_diario,dias_devengo,saldo_insoluto_actual
33991,9988,1,2020-09-15,2440.5691,1809.4309,4250.0,2025-11-10,operativo,195642.4309,USD,...,0.0000,4211.5289,CANCELADA,CANCELADA,CUADRADO,AL DÍA,0.0,60.314363,0,0.0000
33992,9988,2,2020-10-15,2462.8630,1787.1370,4250.0,2025-11-10,operativo,193179.5679,USD,...,0.0000,4211.5289,CANCELADA,CANCELADA,CUADRADO,AL DÍA,0.0,59.571233,0,0.0000
33993,9988,3,2020-11-15,2485.3605,1764.6395,4250.0,2025-11-10,operativo,190694.2074,USD,...,0.0000,4211.5289,CANCELADA,CANCELADA,CUADRADO,AL DÍA,0.0,58.821317,0,0.0000
33994,9988,4,2020-12-15,2508.0635,1741.9365,4250.0,2025-11-10,operativo,188186.1439,USD,...,0.0000,4211.5289,CANCELADA,CANCELADA,CUADRADO,AL DÍA,0.0,58.064550,0,0.0000
33995,9988,5,2021-01-15,2530.9740,1719.0260,4250.0,2025-11-10,operativo,185655.1699,USD,...,0.0000,4211.5289,CANCELADA,CANCELADA,CUADRADO,AL DÍA,0.0,57.300867,0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34047,9988,57,2025-05-15,4061.0955,188.9045,4250.0,2025-11-10,operativo,16618.7557,USD,...,0.0000,4211.5289,CANCELADA,CANCELADA,CUADRADO,AL DÍA,0.0,6.296817,0,0.0000
34048,9988,58,2025-06-15,4098.1925,151.8075,4250.0,2025-11-10,operativo,12520.5632,USD,...,0.0000,4211.5289,CANCELADA,CANCELADA,CUADRADO,AL DÍA,0.0,5.060250,0,0.0000
34049,9988,59,2025-07-15,4135.6283,114.3717,4250.0,2025-11-10,operativo,8384.9349,USD,...,0.0000,4211.5289,CANCELADA,CANCELADA,CUADRADO,AL DÍA,0.0,3.812390,0,0.0000
34050,9988,60,2025-08-15,4173.4060,76.5940,4250.0,2025-11-10,operativo,4211.5289,USD,...,0.0000,4211.5289,CANCELADA,CANCELADA,CUADRADO,AL DÍA,0.0,2.553133,0,4211.5289


: 

In [7]:
df_monto_actual

,operacion,saldo_insoluto_actual,dias_devengo,interes_diario,suma_cuotas_vencida,monto_actual_con_devengo
0,9988,4211.5289,0.0,0.000000,4250.0000,8461.528900
1,9989,9713.8880,1.0,2.232227,0.0000,9716.120227
2,10012,74531.9600,1.0,21.738333,0.0000,74553.698333
3,10443,880.6718,11.0,0.662677,193.9600,1081.921243
4,10447,1638.8220,11.0,1.418413,866.0401,2520.464647
...,...,...,...,...,...,...
478,401122,1901.6733,1.0,0.765167,89.0410,1991.479467
479,401135,172.9931,21.0,0.074483,0.0000,174.557250
480,401138,172.9345,21.0,0.074457,0.0000,174.498090
481,401152,498.3564,21.0,0.200177,24.1636,526.723710


In [8]:
#df_monto_actual[df_monto_actual['operacion']==400915]

In [9]:
#df_filtrado[df_filtrado['operacion']==9988]

In [10]:
#df_monto_actual[df_monto_actual['operacion']==10517]

In [11]:
#df_filtrado[df_filtrado['operacion']==10517].tail(20)

#df_monto_actual[df_monto_actual['operacion']==10517]

# funcion para definir la mora

In [12]:
def construccion_mora(df_filtrado, fecha_reporte, valor_usd, valor_uf):
    """
    Construcción de mora según estándar CMF (RDC01):
    - MONTO AL DÍA = solo cuotas con estado 'AL DÍA' y vencimiento > fecha de reporte.
    - TRAMOS 1–9 = valor_cuota completo de cuotas morosas.
    """

    fecha_reporte = pd.to_datetime(fecha_reporte)
    resultados = []

    operaciones = df_filtrado["operacion"].unique()

    for op in operaciones:
        dn = df_filtrado[df_filtrado["operacion"] == op].copy()

        # Calcular días de mora
        dn["dias_mora"] = (fecha_reporte - dn["fecha_venc"]).dt.days
        dn["dias_mora"] = dn["dias_mora"].clip(lower=0)

        # === MONTO AL DÍA ===
        al_dia = dn[
            (dn["fecha_venc"] > fecha_reporte) &
            (dn["estado_mora"] == "AL DÍA")
        ]
        monto_al_dia = al_dia["valor_cuota"].sum()

        # === CUOTAS MOROSAS ===
        vencidas = dn[
            (dn["fecha_venc"] < fecha_reporte) &
            (dn["estado_mora"] == "MOROSA")
        ]

        # === TRAMOS DE MORA CMF ===
        tramos_def = {
            "Tramo_1": (1, 30),
            "Tramo_2": (30, 60),
            "Tramo_3": (60, 90),
            "Tramo_4": (90, 180),
            "Tramo_5": (180, 365),
            "Tramo_6": (365, 730),
            "Tramo_7": (730, 1095),
            "Tramo_8": (1095, 1460),
            "Tramo_9": (1460, float("inf")),
        }

        mora_tramos = {}
        for tramo, (ini, fin) in tramos_def.items():
            filtro = (vencidas["dias_mora"] >= ini) & (vencidas["dias_mora"] < fin)
            mora_tramos[tramo] = vencidas.loc[filtro, "valor_cuota"].sum()

        # === ÚLTIMA CUOTA VIGENTE ===
        cuotas_vigentes = dn[dn["fecha_venc"] <= fecha_reporte].sort_values("fecha_venc", ascending=False)
        if not cuotas_vigentes.empty:
            ultima_cuota_valor = cuotas_vigentes.iloc[0]["valor_cuota"]
        else:
            futura = dn[dn["fecha_venc"] > fecha_reporte]
            ultima_cuota_valor = futura["valor_cuota"].iloc[0] if not futura.empty else 0

        resultados.append({
            "operacion": op,
            "moneda": dn["moneda"].iloc[0],
            "origen": dn["origen"].iloc[0],
            "valor_cuota": ultima_cuota_valor,
            "dias_mora": vencidas["dias_mora"].max() if not vencidas.empty else 0,
            "Monto_al_dia": monto_al_dia,
            **mora_tramos
        })

    mora = pd.DataFrame(resultados)

    # === CONVERSIÓN A CLP ===
    columnas_conversion = [
        "valor_cuota", "Monto_al_dia",
        "Tramo_1", "Tramo_2", "Tramo_3", "Tramo_4",
        "Tramo_5", "Tramo_6", "Tramo_7", "Tramo_8", "Tramo_9","valor_cuota_cmf","Leasing_Monto_original"
    ]

    

    mora["valor_cuota_cmf"] = mora["valor_cuota"]
    # MONTO ORIGINAL DE LA OPERACIÓN para leasing
    mora["Leasing_Monto_original"] = (
        mora["Monto_al_dia"] +
        mora["Tramo_1"] + mora["Tramo_2"] + mora["Tramo_3"] +
        mora["Tramo_4"] + mora["Tramo_5"] + mora["Tramo_6"] +
        mora["Tramo_7"] + mora["Tramo_8"] + mora["Tramo_9"]
    )

    # Agregar columna 'monto_actual_con_devengo' al DataFrame 'mora'
    mora = mora.merge(
    df_monto_actual[["operacion", "monto_actual_con_devengo"]],
    on="operacion",
    how="left"
    )

    mora.loc[mora["moneda"] == "CLP", columnas_conversion] *= 1
    mora.loc[mora["moneda"] == "USD", columnas_conversion] *= valor_usd
    mora.loc[mora["moneda"] == "UF", columnas_conversion] *= valor_uf

    return mora

In [13]:
#fecha_reporte = "2025-11-14"
mora = construccion_mora(df_filtrado, fecha_reporte, valor_usd, valor_uf)

#mora[mora['origen']== 'financiero']

In [14]:
#df_filtrado[df_filtrado['operacion']==401122].head(20)
#df_monto_actual[df_monto_actual['operacion']==401122]
#mora[mora['operacion']== 401122]

# Seguros


In [15]:
#Esta seccion es unica para calcular las columnas de garantias del RDC01.

#ruta_base_garantia = Path.home() / 'EF Securitizadora' / 'Administracion Maestra - Documentos' / 'adm_maestra'
#archivo_garantia = ruta_base_garantia / 'homologados' / 'incofin' / '6_financiero_garantias' / '6_financiero_garantias_20251020.csv'

#df_garantia = pd.read_csv(archivo_garantia, sep=';', encoding='utf-8')  # puedes cambiar sep=',' si no usa punto y coma,


df_garantia = pd.read_excel('6 financiero garantias_20251031.xlsx')

# Eliminar la coma y la parte decimal ",0", luego convertir a entero para la columna operacion.
#df_garantia['operacion'] = df_garantia['operacion'].str.replace(r',.*', '', regex=True).astype(int)


# Crear la columna 'Garantia_inmobiliaria'.
df_garantia['Garantia_inmobiliaria'] = df_garantia['descripcion'].str.contains(
    r'\b(terreno|TERRENO |departamento|casa|PROPIEDAD|Oficina|inmueble)\b', case=False, na=False
).map({True: 'SI', False: 'NO'})


# Crear la columna 'Garantia_mobiliaria'.
df_garantia['Garantia_mobiliaria'] = df_garantia['Garantia_inmobiliaria'].map({'SI': 'NO', 'NO': 'SI'})

#eliminamos las columnas que no son de nuestro interes.
df_garantia = df_garantia.drop(columns=['tipoGarantia', 'montoTasacionorigonal', 'fechaTasacionOriginal', 'estado'])



# 1️⃣ Crear columna con factor de conversión según moneda
df_garantia["factor_moneda"] = df_garantia["Moneda valorizacion bien"].map({
    "UF": valor_uf,
    "USD": valor_usd,
    "CLP": 1
})

# 2️⃣ Calcular los valores de garantía según tipo
df_garantia["valor_Garantia_inmobiliaria"] = df_garantia.apply(
    lambda x: x["valor"] * x["factor_moneda"] if x["Garantia_inmobiliaria"] == "SI" else 0, axis=1
)

df_garantia["valor_Garantia_mobiliaria"] = df_garantia.apply(
    lambda x: x["valor"] * x["factor_moneda"] if x["Garantia_mobiliaria"] == "SI" else 0, axis=1
)

# 3️⃣ (Opcional) Eliminar la columna auxiliar
# df_garantia = df_garantia.drop(columns=["factor_moneda"])

# --- Resultado final ---
#print(df_garantia[["operacion", "Moneda valorizacion bien", "valor", "factor_moneda","Garantia_inmobiliaria", "valor_Garantia_inmobiliaria","Garantia_mobiliaria", "valor_Garantia_mobiliaria"]].head())

df_garantia.head(20)

C:\Users\MarceloVidal\AppData\Local\Temp\ipykernel_28920\2904665398.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_garantia['Garantia_inmobiliaria'] = df_garantia['descripcion'].str.contains(


,operacion,descripcion,valor,Moneda valorizacion bien,Garantia_inmobiliaria,Garantia_mobiliaria,factor_moneda,valor_Garantia_inmobiliaria,valor_Garantia_mobiliaria
0,400345,"UNA CAMIONETA MARCA TOYOTA, MODELO HILUX DOBLE...",314.61,USD,NO,SI,925.63,0.000000e+00,2.912125e+05
1,400715,CAMIONETA MARCA TOYOTA MODELO HILUX 4X4 2.4L M...,503.35,USD,NO,SI,925.63,0.000000e+00,4.659159e+05
2,294602,"INMUEBLE COMERCIAL UBICADO EN LA CRUZ 540, RAN...",8576.47,USD,SI,NO,925.63,7.938638e+06,0.000000e+00
3,400544,ALZA CONTENEDOR PERKINS D6213,9500.00,USD,NO,SI,925.63,0.000000e+00,8.793485e+06
4,298901,SWITCHES,10891.33,USD,NO,SI,925.63,0.000000e+00,1.008134e+07
5,400738,carrocerÃ­a marca tremac Carroceria forestal B...,13322.00,USD,NO,SI,925.63,0.000000e+00,1.233124e+07
6,298901,SERVIDORES,16464.46,USD,NO,SI,925.63,0.000000e+00,1.524000e+07
7,298901,POWERVAULT,20702.10,USD,NO,SI,925.63,0.000000e+00,1.916248e+07
8,400416,GRUA HORQUILLA MARCA YALE MODELO GTP 30 AÃ‘O 2...,23000.00,USD,NO,SI,925.63,0.000000e+00,2.128949e+07
9,401168,Tipo de Bien: (Maquina Industrial) Miniexcavad...,24100.00,USD,NO,SI,925.63,0.000000e+00,2.230768e+07


In [16]:
#0 origen
#R: df_cartera['origen']

#1.RUT
#R: df_cartera['rut_id']
#2.TIPO PERSONA
#R: df_cartera['TIPO_PERSONA']
#3.CÓDIGO OPERACIÓN
#R: df_cartera['	operacion']
#4.OPERACIÓN TÍTULO III
#R: 1


#5.TIPO DE DEUDOR
#R: tipo de deudor, directo === 1 


#6.TIPO DE CRÉDITO
#R:
#Leasing consumo --> 31 si df_cartera['TIPO_PERSONA']==  1 (personas naturales)
#Leasing comercial --> 33 si df_cartera['TIPO_PERSONA']== 2 (para persona juridica)


#7.FECHA DE OTORGAMIENTO
#R: df_cartera['inicio']

#8.CARGA FINANCIERA: valor cuota
#R: mora['valor_cuota_cmf']

#9.FECHA DE EXTINCIÓN DE LA OPERACIÓN
#R: df_cartera['termino']




#10. VALOR DE LA GARANTÍA REAL INMOBILIARIA clp
#R: df_garantia['valor_Garantia_inmobiliaria']

#11.VALOR DE LA GARANTÍA REAL MOBILIARIA clp
#R: df_garantia['valor_Garantia_mobiliaria']

#12. VALOR DE LA GARANTÍA FINANCIERA
#R:0
#13.VALOR DE LA GARANTÍA AVAL O FIANZA
#R:0

#14.MONTO ORIGINAL DE LA OPERACIÓN clp
#R: df_cartera['saldo_inicial_clp']

#15.MONTO ACTUAL DE LA OPERACIÓN clp
#R: df_cartea['saldo_cap_clp']

#16.MONTO AL DÍA clp
#R: mora['Monto_al_dia']

#17.MONTO MORA 1er TRAMO clp
#R: mora['Tramo_1']
#18.MONTO MORA 2do TRAMO clp
#R:mora['Tramo_2']
#19.MONTO MORA 3er TRAMO clp
#R:mora['Tramo_3']
#20.MONTO MORA 4to TRAMO clp
#R:mora['Tramo_4']
#21.MONTO MORA 5to TRAMO clp
#R:mora['Tramo_5']
#22. MONTO MORA 6to TRAMO clp
#R:mora['Tramo_6']
#23.MONTO MORA 7mo TRAMO clp
#R:mora['Tramo_7']
#24.MONTO MORA 8vo TRAMO clp
#R:mora['Tramo_8']
#25.MONTO MORA 9no TRAMO clp
#R:mora['Tramo_9']
#26.MORA ACTUAL ->dias
#R:mora['dias_mora']


#27.DEUDA RENEGOCIADA
#R:2
#28.DEUDA ACELERADA
#R:2


In [17]:
# ===============================
# 🧩 1. Asegurar tipos consistentes
# ===============================
df_cartera["operacion"] = df_cartera["operacion"].astype(int)
mora["operacion"] = mora["operacion"].astype(int)
df_garantia["operacion"] = df_garantia["operacion"].astype(int)

# ===============================
# 🧮 2. Consolidar MORA (una fila por operación)
# ===============================
cols_sum_mora = [
    "valor_cuota", "valor_cuota_cmf", "Monto_al_dia",
    "Tramo_1", "Tramo_2", "Tramo_3", "Tramo_4", "Tramo_5",
    "Tramo_6", "Tramo_7", "Tramo_8", "Tramo_9","Leasing_Monto_original","monto_actual_con_devengo"
]

mora_agg = (
    mora.groupby("operacion", as_index=False)
        .agg({**{c: "sum" for c in cols_sum_mora},
              "dias_mora": "max",
              "moneda": "first",
              "origen": "first"})
)

# ===============================
# 🏦 3. Consolidar GARANTÍA (una fila por operación)
# ===============================
cols_sum_gar = ["valor_Garantia_inmobiliaria", "valor_Garantia_mobiliaria"]

df_garantia_agg = (
    df_garantia.groupby("operacion", as_index=False)
               .agg({**{c: "sum" for c in cols_sum_gar},
                     "descripcion": "first",
                     "Moneda valorizacion bien": "first"})
)

# ===============================
# 🔗 4. Merge garantizando todas las operaciones de df_cartera
# ===============================
df_merge = (
    df_cartera
    .merge(mora_agg, on="operacion", how="left", suffixes=("", "_mora"))
    .merge(df_garantia_agg, on="operacion", how="left", suffixes=("", "_gar"))
    .fillna({
        # Rellenar garantías ausentes con 0
        "valor_Garantia_inmobiliaria": 0,
        "valor_Garantia_mobiliaria": 0,
        # Rellenar valores de mora ausentes con 0
        "valor_cuota_cmf": 0,
        "Monto_al_dia": 0,
        "Tramo_1": 0, "Tramo_2": 0, "Tramo_3": 0, "Tramo_4": 0, "Tramo_5": 0,
        "Tramo_6": 0, "Tramo_7": 0, "Tramo_8": 0, "Tramo_9": 0,
        "dias_mora": 0
    })
)

# ===============================
# 🧱 5. Construcción del DataFrame final
# ===============================
prueba_RDC01 = pd.DataFrame({
    "origen": df_merge["origen"],
    "RUT": df_merge["rut_id"],
    "TIPO_PERSONA": df_merge["TIPO_PERSONA"],
    "CODIGO_OPERACION": df_merge["operacion"],
    "OPERACION_TITULO_III": 4,
    "TIPO_DEUDOR": 1,
    "TIPO_CREDITO": np.where(df_merge["TIPO_PERSONA"] == 1, 31, 33),
    "FECHA_OTORGAMIENTO": df_merge["inicio"],
    "CARGA_FINANCIERA": df_merge["valor_cuota_cmf"],#*1.19,
    "FECHA_EXTINCION_OPERACION": df_merge["termino"],
    "VALOR_GARANTIA_INMOBILIARIA": df_merge["valor_Garantia_inmobiliaria"],
    "VALOR_GARANTIA_MOBILIARIA": df_merge["valor_Garantia_mobiliaria"],
    "VALOR_GARANTIA_FINANCIERA": 0,
    "VALOR_GARANTIA_AVAL_FIANZA": 0,
    "MONTO_ORIGINAL_OPERACION": df_merge["Leasing_Monto_original"],#*1.19,
    "MONTO_ACTUAL_OPERACION": df_merge["monto_actual_con_devengo"],#*1.19,
    "MONTO_AL_DIA": df_merge["Monto_al_dia"],#*1.19,
    "MONTO_MORA_1_TRAMO": df_merge["Tramo_1"],#*1.19,
    "MONTO_MORA_2_TRAMO": df_merge["Tramo_2"],#*1.19,
    "MONTO_MORA_3_TRAMO": df_merge["Tramo_3"],#*1.19,
    "MONTO_MORA_4_TRAMO": df_merge["Tramo_4"],#*1.19,
    "MONTO_MORA_5_TRAMO": df_merge["Tramo_5"],#*1.19,
    "MONTO_MORA_6_TRAMO": df_merge["Tramo_6"],#*1.19,
    "MONTO_MORA_7_TRAMO": df_merge["Tramo_7"],#*1.19,
    "MONTO_MORA_8_TRAMO": df_merge["Tramo_8"],#*1.19,
    "MONTO_MORA_9_TRAMO": df_merge["Tramo_9"],#*1.19,
    "MORA_ACTUAL": df_merge["dias_mora"],
    "DEUDA_RENEGOCIADA": 2,
    "DEUDA_ACELERADA": 2
})

# ===============================
# 🚫 6. Eliminar filas con origen = 0
# ===============================
prueba_RDC01 = prueba_RDC01[
    prueba_RDC01["origen"].astype(str).str.strip() != "0"
].copy()

# ===============================
# 📊 7. Verificación
# ===============================
print("✅ DataFrame prueba_RDC01 generado correctamente.")
print("Tamaño df_cartera:", df_cartera.shape)
print("Tamaño prueba_RDC01:", prueba_RDC01.shape)
print("Coinciden en cantidad de operaciones:",
      df_cartera["operacion"].nunique() == prueba_RDC01["CODIGO_OPERACION"].nunique())

prueba_RDC01['origen'].unique()

✅ DataFrame prueba_RDC01 generado correctamente.
Tamaño df_cartera: (618, 25)
Tamaño prueba_RDC01: (571, 29)
Coinciden en cantidad de operaciones: False


array(['financiero', 'operativo'], dtype=object)

In [18]:
#prueba_RDC01[prueba_RDC01['CODIGO_OPERACION']==401122]

In [19]:
#df_cartera[df_cartera['operacion']== 401122]

In [20]:
#df_filtrado[df_filtrado['operacion']==401122].head()


In [21]:
#df_monto_actual[df_monto_actual['operacion']==401122]

In [22]:
#mora[mora['operacion']== 401122]

In [23]:
# Eliminar filas donde origen == 0
prueba_RDC01 = prueba_RDC01[prueba_RDC01["origen"] != 0].copy()

# Verificar resultado
filas, columnas = prueba_RDC01.shape
print(f"✅ Se eliminaron las filas con origen = 0. El archivo ahora tiene {filas} registros y {columnas} columnas.")

✅ Se eliminaron las filas con origen = 0. El archivo ahora tiene 571 registros y 29 columnas.


In [24]:
# === Crear reporte_RDC01 solo con origen financiero ===
reporte_RDC01 = (
    prueba_RDC01[prueba_RDC01["origen"].astype(str).str.strip().str.lower() == "financiero"]
    .drop(columns=["origen"])
    .copy()
)

# === Eliminar la operación específica ===
#reporte_RDC01 = reporte_RDC01[reporte_RDC01["CODIGO_OPERACION"] != 400111].copy()

# === Confirmar resultado ===
filas, columnas = reporte_RDC01.shape

print(f"   Registros actuales: {filas} | Columnas: {columnas}")

cond = (
    (reporte_RDC01['CODIGO_OPERACION'] == 400111) &
    (reporte_RDC01['RUT'].isna() | reporte_RDC01['RUT'].str.strip().isin([None, '', 'nan']))
)

reporte_RDC01.loc[cond, 'RUT'] = '771976573'
## el codigo de operacion 400111 no tiene registrado el rut, por lo cual se debio hacer de manerea manual.

#reporte_RDC01[reporte_RDC01['CODIGO_OPERACION']== 400111]

   Registros actuales: 477 | Columnas: 28


In [25]:
## observaremos si tenememos un valor NaN dentro de nuestras columnas
reporte_RDC01.isna().sum()

RUT                            0
TIPO_PERSONA                   0
CODIGO_OPERACION               0
OPERACION_TITULO_III           0
TIPO_DEUDOR                    0
TIPO_CREDITO                   0
FECHA_OTORGAMIENTO             0
CARGA_FINANCIERA               0
FECHA_EXTINCION_OPERACION      0
VALOR_GARANTIA_INMOBILIARIA    0
VALOR_GARANTIA_MOBILIARIA      0
VALOR_GARANTIA_FINANCIERA      0
VALOR_GARANTIA_AVAL_FIANZA     0
MONTO_ORIGINAL_OPERACION       0
MONTO_ACTUAL_OPERACION         0
MONTO_AL_DIA                   0
MONTO_MORA_1_TRAMO             0
MONTO_MORA_2_TRAMO             0
MONTO_MORA_3_TRAMO             0
MONTO_MORA_4_TRAMO             0
MONTO_MORA_5_TRAMO             0
MONTO_MORA_6_TRAMO             0
MONTO_MORA_7_TRAMO             0
MONTO_MORA_8_TRAMO             0
MONTO_MORA_9_TRAMO             0
MORA_ACTUAL                    0
DEUDA_RENEGOCIADA              0
DEUDA_ACELERADA                0
dtype: int64

In [26]:
#EXPORTAR EL DATAFRAME A EXCEL y duirigirlo a la ruta correspondiente
fecha_suffix = pd.to_datetime(fecha_reporte).strftime("%Y%m%d")

nombre_archivo = f"RDC01_INCOFIN_{fecha_suffix}.xlsx"

# Ruta destino local
ruta_destino = Path(r"C:\Users\MarceloVidal\EF Securitizadora\Operaciones - Documentos\Documentos\Operaciones\REDEC\INCOFIN_ACFIN\archivo_xlsx")

archivo_completo = ruta_destino / nombre_archivo

# Guardar
reporte_RDC01.to_excel(archivo_completo, index=False)


In [27]:
#caratula_texto
#numero de registros
print('numero de registros',len(reporte_RDC01['RUT']))
print('numero de deudores ',len(reporte_RDC01['RUT'].unique()))
print('reportable al dia', reporte_RDC01['MONTO_AL_DIA'].sum())
print('mora tramo 1', reporte_RDC01['MONTO_MORA_1_TRAMO'].sum())
print('mora tramo 2', reporte_RDC01['MONTO_MORA_2_TRAMO'].sum())
print('mora tramo 3', reporte_RDC01['MONTO_MORA_3_TRAMO'].sum())
print('mora tramo 4', reporte_RDC01['MONTO_MORA_4_TRAMO'].sum())
print('mora tramo 5', reporte_RDC01['MONTO_MORA_5_TRAMO'].sum())
print('mora tramo 6', reporte_RDC01['MONTO_MORA_6_TRAMO'].sum())
print('mora tramo 7', reporte_RDC01['MONTO_MORA_7_TRAMO'].sum())
print('mora tramo 8', reporte_RDC01['MONTO_MORA_8_TRAMO'].sum())
print('mora tramo 9', reporte_RDC01['MONTO_MORA_9_TRAMO'].sum())
print('numero de deudores', (reporte_RDC01['MORA_ACTUAL'] > 0).sum() )
print('total lineas de credito de libre disposicion = 0')



numero de registros 477
numero de deudores  236
reportable al dia 16334470059.241772
mora tramo 1 535710912.7997149
mora tramo 2 210125304.348433
mora tramo 3 51108688.387111
mora tramo 4 7368185.387913
mora tramo 5 0.0
mora tramo 6 0.0
mora tramo 7 0.0
mora tramo 8 0.0
mora tramo 9 0.0
numero de deudores 219
total lineas de credito de libre disposicion = 0


In [28]:
#caratula_texto = generar_caratula_rdc01(reporte_RDC01, codigo_institucion="0000000782", fecha_reporte="2025-11-07")

In [29]:
from datetime import datetime

def exportar_rdc01_txt(df, codigo_institucion="0000000782", fecha_reporte=None, nombre_base="RDC01"):
    """
    Exporta un DataFrame al formato TXT del archivo RDC01 (CMF).
    - df: DataFrame con columnas del formato RDC01
    - codigo_institucion: código CMF de la institución (ej: '0000000782')
    - fecha_reporte: fecha del archivo (por defecto: fecha actual)
    """
    # --- 1️⃣ Preparar parámetros ---
    if fecha_reporte is None:
        fecha_reporte = datetime.now().strftime("%Y%m%d")
    else:
        fecha_reporte = pd.to_datetime(fecha_reporte).strftime("%Y%m%d")

    nombre_archivo = f"{nombre_base}_{codigo_institucion}_{fecha_reporte}.TXT"

    # --- 2️⃣ Ordenar columnas según formato RDC01 ---
    columnas_rdc01 = [
        "RUT", "TIPO_PERSONA", "CODIGO_OPERACION", "OPERACION_TITULO_III",
        "TIPO_DEUDOR", "TIPO_CREDITO", "FECHA_OTORGAMIENTO", "CARGA_FINANCIERA",
        "FECHA_EXTINCION_OPERACION", "VALOR_GARANTIA_INMOBILIARIA", "VALOR_GARANTIA_MOBILIARIA",
        "VALOR_GARANTIA_FINANCIERA", "VALOR_GARANTIA_AVAL_FIANZA", "MONTO_ORIGINAL_OPERACION",
        "MONTO_ACTUAL_OPERACION", "MONTO_AL_DIA", "MONTO_MORA_1_TRAMO", "MONTO_MORA_2_TRAMO",
        "MONTO_MORA_3_TRAMO", "MONTO_MORA_4_TRAMO", "MONTO_MORA_5_TRAMO", "MONTO_MORA_6_TRAMO",
        "MONTO_MORA_7_TRAMO", "MONTO_MORA_8_TRAMO", "MONTO_MORA_9_TRAMO",
        "MORA_ACTUAL", "DEUDA_RENEGOCIADA", "DEUDA_ACELERADA"
    ]
    df_txt = df[columnas_rdc01].copy()

    # --- 3️⃣ Formatear fechas ---
    for col in ["FECHA_OTORGAMIENTO", "FECHA_EXTINCION_OPERACION"]:
        df_txt[col] = pd.to_datetime(df_txt[col], errors="coerce").dt.strftime("%Y%m%d").fillna("00000000")

    # --- 4️⃣ Formatear numéricos ---
    cols_numericas = [
        "CARGA_FINANCIERA", "VALOR_GARANTIA_INMOBILIARIA", "VALOR_GARANTIA_MOBILIARIA",
        "VALOR_GARANTIA_FINANCIERA", "VALOR_GARANTIA_AVAL_FIANZA", "MONTO_ORIGINAL_OPERACION",
        "MONTO_ACTUAL_OPERACION", "MONTO_AL_DIA", "MONTO_MORA_1_TRAMO", "MONTO_MORA_2_TRAMO",
        "MONTO_MORA_3_TRAMO", "MONTO_MORA_4_TRAMO", "MONTO_MORA_5_TRAMO", "MONTO_MORA_6_TRAMO",
        "MONTO_MORA_7_TRAMO", "MONTO_MORA_8_TRAMO", "MONTO_MORA_9_TRAMO"
    ]
    for c in cols_numericas:
        df_txt[c] = df_txt[c].fillna(0).astype(float).round(0).astype(int).astype(str).str.zfill(15)

    # --- 5️⃣ Otras columnas ---
    df_txt["TIPO_PERSONA"] = df_txt["TIPO_PERSONA"].astype(int).astype(str)
    df_txt["OPERACION_TITULO_III"] = df_txt["OPERACION_TITULO_III"].astype(int).astype(str)
    df_txt["TIPO_DEUDOR"] = df_txt["TIPO_DEUDOR"].astype(int).astype(str)
    df_txt["TIPO_CREDITO"] = df_txt["TIPO_CREDITO"].astype(int).astype(str).str.zfill(2)
    df_txt["MORA_ACTUAL"] = df_txt["MORA_ACTUAL"].fillna(0).astype(int).astype(str).str.zfill(4)
    df_txt["DEUDA_RENEGOCIADA"] = df_txt["DEUDA_RENEGOCIADA"].astype(int).astype(str)
    df_txt["DEUDA_ACELERADA"] = df_txt["DEUDA_ACELERADA"].astype(int).astype(str)
    df_txt["CODIGO_OPERACION"] = df_txt["CODIGO_OPERACION"].astype(str).str.strip().str.zfill(30)
    df_txt["RUT"] = df_txt["RUT"].astype(str).str.replace(r"[^0-9]", "", regex=True).str.zfill(10)

    # --- 6️⃣ Generar encabezado del archivo ---
    header_line = f"{codigo_institucion}RDC01{fecha_reporte}".ljust(322, " ")

    # --- 7️⃣ Generar líneas del archivo ---
    txt_lines = df_txt.apply(lambda row: "".join(row.astype(str)), axis=1)

    # --- 8️⃣ Escribir archivo TXT ---
    with open(nombre_archivo, "w", encoding="utf-8") as f:
        f.write(header_line + "\n")
        for line in txt_lines:
            f.write(line + "\n")

    print(f"✅ Archivo TXT generado correctamente: {nombre_archivo}")
    print(f"📦 Registros exportados: {len(txt_lines)}")

    return nombre_archivo

In [30]:
# Exportar el DataFrame reporte_RDC01
#exportar_rdc01_txt(reporte_RDC01, codigo_institucion="0000000782", fecha_reporte="2025-11-07")